Wczytanie zbioru danych

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/My Drive/data_sety/ready_data_set.csv'
data_set = pd.read_csv(file_path)
print(data_set.head())

   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                     normalized_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  t

Instalacja i importowanie bibliotek

In [3]:
!pip install transformers

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch



Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2

Sprawdzenie, czy PyTorch korzysta z GPU

In [4]:
if torch.cuda.is_available():
    print("CUDA is available. GPU is ready for use.")
    print("CUDA version:", torch.version.cuda)
else:
    print("CUDA is not available. Check your installation.")


CUDA is available. GPU is ready for use.
CUDA version: 12.1


Inicjalizacja tokenizera i modelu BERT.
Wybieramy odpowiednią wersję BERTa. Dla języka angielskiego - bert-base-uncased.
Następnie przeniesienie modelu na GPU, żeby maksymalnie wykorzystać T4 GPU

P

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

Osadzanie tekstów w całym zbiorze danych. Aby zwiększyć wydajnośc, tekst przetwarzanmy partiami.

In [6]:
def bert_embedding(sentences, model, tokenizer):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt').to('cuda')
    with torch.no_grad():
        outputs = model(**encoded_input)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()  # Zwróć wyniki na CPU
    return embeddings

# Przetwarzaj teksty w partiami
def batch_process(texts, batch_size=32):
    batched_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        embeddings = bert_embedding(batch_texts, model, tokenizer)
        batched_embeddings.append(embeddings)
    return np.vstack(batched_embeddings)

bert_embeddings = batch_process(data_set['text'].tolist())


Normalizacja wektorów osadzeń do normy euklidesowej - Dzięki temu różnice w długości wektorów nie wpłyną na wyniki algorytmów klasteryzacji

Po normalizacji, punkty są skalowane w taki sposób, by ich odległość od początku układu współrzędnych wynosiła 1.

In [7]:
from sklearn.preprocessing import normalize
normalized_embeddings = normalize(bert_embeddings, norm='l2', axis=1)

Usuwanie outlierów - obserwacji, które znacząco odbiegają od pozostałych danych i mogą wpływać na wyniki klasteryzacji

In [8]:
from scipy import stats
bert_embeddings_df = pd.DataFrame(normalized_embeddings)
z_scores = np.abs(stats.zscore(bert_embeddings_df))
threshold = 3
filtered_bert_embeddings_df = bert_embeddings_df[(z_scores < threshold).all(axis=1)]

Zapisanie zbioru gotowego do klasteryzacji

In [9]:
filtered_bert_embeddings_df.to_csv('/content/drive/My Drive/data_sety/normalized_filtered_bert_embeddings.csv', index=False)